Pytorch 모듈은 텐서가 CxHxW (채널x높이x너비) 순으로 배치 되어야 한다.

In [1]:
import imageio

img_arr = imageio.v2.imread('../data/image/bobby.jpg')
img_arr.shape

(720, 1280, 3)

In [2]:
import torch
# 순서 바꾸기 이 연산은 텐서 복사본을 만들지 않는다.
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)
print(out.size())
print(img.storage_offset() == out.storage_offset())

torch.Size([3, 720, 1280])
True


batch image >> NxCxHxW

In [3]:
batch_size = 3
batch = torch.zeros(batch_size, 3 ,256, 256, dtype=torch.uint8)

In [4]:
import os

data_dir = '../data/image/'
# cat(n).png만 가져오기
filenames = [name for name in os.listdir(data_dir)
             if name.startswith('cat')]
filenames

['cat1.png', 'cat2.png', 'cat3.png']

In [5]:
for i, filename in enumerate(filenames):
    img_arr = imageio.v2.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2,0,1)
    img_t = img_t[:3] # 이미지 중에는 RBG말고 다른 값들도 있는 것도 있지만 우리는 RGB만 사용하므로
    batch[i] = img_t

batch.size()

torch.Size([3, 3, 256, 256])

데이터 정규화

In [6]:
n_channels = batch.shape[1]
# 텐서 계산은 기본이 float
batch = batch.float()
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

batch[0].shape

torch.Size([3, 256, 256])

In [7]:
dir_path = '../data/dicom'
vol_arr = imageio.v2.volread(dir_path,'DICOM')
vol_arr.shape

Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 99/99  (100.0%)


(99, 512, 512)

NxCxDxHxW (D는 depth) 하지만, 위에를 보면 채널과 batch_num이 빠져있다.   
채널은 흑백 1채널로 1이다.

In [8]:
vol = torch.from_numpy(vol_arr).float()
vol = torch.unsqueeze(vol, 0)

vol.shape

torch.Size([1, 99, 512, 512])